In [1]:
import numpy as np
import main as mc
import matplotlib.pyplot as plt

In [2]:
def backTest(net, X, Y, score=2):
    # Test model as buy indicator
    moneys = 0
    buys = 0
    correct = 0
    for day in range(len(X)):
        # Get prediction of next open
        nextPrediction = net.Calculate(X[day], useFast=True)

        # Compare prediction to current closing val
        ## (Assume buying near end of day so "current close" ~= real close)
        currentClose = X[day, 7]

        # Do buy check
        nextReal = Y[day]
        if nextPrediction > currentClose:
            change = nextReal - currentClose
            moneys += change
            buys += 1
            if change > 0:
                correct += 1
    
    if buys == 0:
        buys = 1
        moneys = 1
        correct = 1
        
    if score == 0:
        return moneys
    elif score == 1:
        return correct/buys
    elif score == 2:
        return (correct/buys) * (moneys)

In [3]:
import yfinance as yf

spy = yf.download("SPY", "2000-01-01")
spy.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,148.25000,148.25000,143.875000,145.43750,94.595062,8164300
2000-01-04,143.53125,144.06250,139.640625,139.75000,90.895859,8089800
2000-01-05,139.93750,141.53125,137.250000,140.00000,91.058441,12177900
2000-01-06,139.62500,141.50000,137.750000,137.75000,89.594986,6227200
2000-01-07,140.31250,145.75000,140.062500,145.75000,94.798340,8066500
2000-01-10,146.25000,146.90625,145.031250,146.25000,95.123520,5741700
2000-01-11,145.81250,146.09375,143.500000,144.50000,93.985313,7503700
2000-01-12,144.59375,144.59375,142.875000,143.06250,93.050369,6907700
2000-01-13,144.46875,145.75000,143.281250,145.00000,94.310524,5158300


In [4]:
print("Hold Results:")
dm = spy.iloc[-1]["Close"] - spy.iloc[0]["Open"]
print(f"Net = ${dm}")
print(f"Avg. Daily = ${dm/len(spy)}")

Hold Results:
Net = $302.94000244140625
Avg. Daily = $0.05087153693390533


In [5]:
X1 = spy.iloc[:-2][["Open", "High", "Low", "Close"]]
X2 = spy.iloc[1:-1][["Open", "High", "Low", "Close"]]
X = np.zeros((len(X1), 8))
X[:, :4] = X1
X[:, 4:] = X2
Y = spy.iloc[2:]["Close"]

XTrain, YTrain, XTest, YTest = mc.TTSplit(X, Y, percentTrain=50)

In [7]:
# Load in a model
net = mc.AdvNet(8, [], 1, ['lin'])
# net = mc.LoadNet("stock256")

# Train model to training data
net = net.FastTrain(XTrain, YTrain, Ieta=5, Beta=100, Gamma=2)

print(f"\nResults on Train Data:")
print(f"R^2: {mc.netMetrics(net, XTrain, YTrain)}")
p1 = backTest(net, XTrain, YTrain, score=0)
print(f"Backtest: ${p1}")
print(f"Average profit/trade = ${round(p1 / len(XTrain), 3)}")

print(f"\nResults on Test Data:")
print(f"R^2: {mc.netMetrics(net, XTest, YTest)}")
p2 = backTest(net, XTest, YTest, score=0)
print(f"Backtest: ${p2}")
print(f"Average profit/trade = ${round(p2 / len(XTest), 3)}")

prof = backTest(net, X, Y, score=0)
acc  = round(100*backTest(net, X, Y, score=1), 2)
print(f"\nNet Accuracy = {acc}%")
print(f"Total Profit = ${round(prof, 2)}")
print(f"Average profit/trade = ${round(prof / len(X), 3)}")

escore = round(acc*prof/100)
print(f"\nExp. Score = {escore}")
if escore > 230:
    net.SaveNN(f"stock{escore}")

Score: 0.999250 | Training: ========================================                   

Results on Train Data:
R^2: 0.9992495339398487
Backtest: $154.82231903076172
Average profit/trade = $0.052

Results on Test Data:
R^2: 0.9992646125988172
Backtest: $82.41969299316406
Average profit/trade = $0.028

Net Accuracy = 54.97%
Total Profit = $237.24
Average profit/trade = $0.04

Exp. Score = 130
